In [1]:
import os, sys

pysbf_path = "/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/"
sys.path.insert(0, pysbf_path)
from pysbf import *

In [2]:
import astropy.io.fits as fits
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab as py

import sqlcl

In [3]:
import astropy.io.fits as pyfits

In [4]:
def xcmd(cmd,verbose):

    if verbose: print('\n'+cmd)

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if False:
        print('execution of %s failed' % cmd)
        print('error is as follows',output)
        sys.exit()
    else:
        return output
    
from contextlib import contextmanager
import os

@contextmanager
def cwd(path):
    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)
        
        
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [5]:
def add_hdr(hdr, header, key):
    
    value = header[key]
    
    if isinstance(value, str):
        value = "'" + value + "'"
    else:
        value = str(value)
    
    hdr += key + "=" + value + " / " + header.comments[key] + "\n"
    
    return hdr

In [6]:
def getPGCid(objname):
    """extracting the PGC ID of a galaxy given its name

    Args:
        objname  (str): galaxy name

    Returns:
        int: PGC ID
    """

    
    my_url = 'http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?objname='+objname+'&extend=no&of=xml_names&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES'
    tag = mySoup(my_url).findAll('a', href="/cgi-bin/catdef?prefix=PGC")[0].parent
    pgc = int(str(tag).split("</a>")[1].split('</td>')[0].strip())
   
    return pgc

In [7]:
def addGalaxyInfo(Leda, pgc, response=None, objname=None):
    """Exctracting the information of a galaxy from the LEDA catalog 
    given the corresponsing PGC ID

    :param Leda: Leda catalog
    :type Leda: ``Pandas dataFrame``
    :param pgc: pgc ID
    :type pgc: ``int``
    :param response: response to be modified, defaults to None
    :type response: python ``dictionary``, optional
    :param objname: galaxy name, defaults to None
    :type objname: ``str``, optional
    :return: response dictionary that holds the requested galaxy information
    :rtype: python dictionary
    """

    if response is None:
        response = {}

    myGalaxy = Leda.loc[int(pgc)]
    response["pgc"]     = pgc
    response["ra"]      = "%.4f"%((myGalaxy.al2000)*15)         # deg
    response["dec"]     = "%.4f"%(myGalaxy.de2000)              # deg
    response["fov"]     = "%.2f"%(1.5*0.1*10**myGalaxy.logd25)  # arcmin
    response["pa"]      = str(myGalaxy.pa)

    if objname is None:
        response["objname"] = str(myGalaxy.objname)
    else:
        response["objname"] = objname

    return response

In [8]:
def loadLeda(leda_catalog):
    """Loading the HyperLeda catalog: 
    - http://leda.univ-lyon1.fr/
    - This catalog tabulates the proper information on local galaxies

    :return: the Leda catalog in the Pandas dataFrame format
    :rtype: Pandas ``dataFrame``
    """

    df = pd.read_csv(leda_catalog, delimiter=',')
    df = df.rename(columns=lambda x: x.strip())
                   
    return df.set_index('PGC')

In [9]:
Leda = loadLeda("LEDA.csv")

Leda.head()

<ipython-input-9-a7ac7fd79853>:1: DtypeWarning: Columns (71,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  Leda = loadLeda("LEDA.csv")


,al1950,de1950,al2000,de2000,l2,b2,sgl,sgb,t,e_t,...,objname,objtype,type,bar,ring,multiple,compactness,angclass,RA2000,DEC2000
PGC,,,,,,,,,,,,,,,,,,,,,
1,23.95695,0.42327,23.99966,0.70168,96.87946,-59.54180,293.35019,13.43002,2.4,5.0,...,PGC000001,G,S?,NaN,NaN,NaN,NaN,NaN,235958.8,4206.0
2,23.95786,46.99616,0.00047,47.27450,113.95535,-14.69916,341.64403,20.73880,3.1,0.4,...,UGC12889,G,Sb,B,NaN,NaN,NaN,NaN,1.7,471628.2
4,23.95829,22.80927,0.00096,23.08764,107.83219,-38.27289,316.05865,18.45140,5.0,1.2,...,PGC000004,G,Sc,NaN,NaN,NaN,NaN,NaN,3.5,230515.5
5,23.95988,32.46003,0.00254,32.73839,110.62060,-28.90431,326.17705,19.78055,-0.6,1.2,...,IC5370,G,S0-a,NaN,NaN,NaN,NaN,NaN,9.1,324418.2
6,23.95790,15.60327,0.00058,15.88165,105.25252,-45.21822,308.62424,17.10314,-1.0,2.0,...,PGC000006,G,S0-a,NaN,NaN,NaN,C,NaN,2.1,155253.9


In [20]:
def load_info(galaxy_name):
    
    ned = get_ned_info([galaxy_name])
    
    try:
        if ned is not None:
            objName = ned.iloc[0]["Object Name"]
            ra, dec = ned.iloc[0]["ra_dec"]

            pgc = getPGCid(objName.replace(" ", "%20"))
            logd25 = Leda.loc[pgc]["logd25"]
            
            return pgc, objName.replace(" ", ""), ra, dec, logd25
    except: 
        print(f"Couldn't load data for {galaxy_name} ... !")
        pass
        
    return None

In [21]:
def fits_prep(filename):

    name = removefix(filename)
    xcmd("cp " + filename + " " + name + ".tmp.fits", True)

    hdulist = pyfits.open(filename)
    prihdr = hdulist[1].header

#     if edit_zp:

#         hdulist = pyfits.open(name + ".tmp.fits", mode="update")
#         prihdr = hdulist[0].header
#         prihdr["BSCALE"] = 1.0
#         prihdr["BZERO"] = 0.0

#         expTime = float(prihdr["EXPTIME"])
#         nMgy = float(prihdr["NMGY"])

#         new_zp = 16.40006562  # mJy / pix

#         ### https://data.sdss.org/datamodel/files/BOSS_PHOTOOBJ/frames/RERUN/RUN/CAMCOL/frame.html
#         ### http://www.sdss.org/dr12/imaging/images/
#         zeroPoint = 22.5
#         alpha = 10 ** (-0.4 * (zeroPoint - new_zp))


#         hdulist[0].data = data = hdulist[0].data * (alpha)

#         prihdr["EXPTIME"] = 1.0
#         prihdr["ZP"] = new_zp

#         hdulist.flush()

    cmd = "mv " + name + ".tmp.fits " + filename
    xcmd(cmd, True)


In [22]:
def getSDSSfiles(fieldInfo, band, folder):

    run = fieldInfo[0]
    camcol = fieldInfo[1]
    field = fieldInfo[2]

    fileName = (
        "frame-" + band + "-"
        "{0:06d}".format(int(run))
        + "-"
        + camcol
        + "-"
        + "{0:04d}".format(int(field))
        + ".fits.bz2"
    )
    filename = (
        "frame-" + band + "-"
        "{0:06d}".format(int(run))
        + "-"
        + camcol
        + "-"
        + "{0:04d}".format(int(field))
        + ".fits"
    )
    http = "https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/"

    http += run + "/"
    http += camcol + "/"
    http += fileName

    with cd(folder):
        xcmd("wget " + http, True)
        xcmd("bzip2 -d " + fileName, True)
        fits_prep(filename)


In [23]:
#################################
def removefix(filename):

    name_list = filename.split(".")
    N = len(name_list)
    if name_list[N - 1] == "fits":

        name = ""
        for i in range(N - 2):
            name = name + name_list[i] + "."
        name += name_list[N - 2]

    return name


#################################





In [24]:
def getSDSSfields(ra, dec, size):  # all in degree

    delta = 6 * size

    # if size > 1:
    # delta = 1.2*size+0.4

    ra_max = ra + 2 * delta
    ra_min = ra - 2 * delta
    dec_max = dec + delta
    dec_min = dec - delta

    querry = """
  
 SELECT
 fieldID,
 run, 
 camCol, 
 field,
 ra, 
 dec,
 run,
 rerun 
 FROM Field   
  """

    querry += (
        "WHERE ra BETWEEN "
        + str(ra_min)
        + " and "
        + str(ra_max)
        + " and dec BETWEEN "
        + str(dec_min)
        + " and "
        + str(dec_max)
    )

    print(querry)

    lines = sqlcl.query(querry).readlines()
    lines = [str(l) for l in lines]

    N = len(lines)

    field_lst = []
    for i in np.arange(2, N):
        line = lines[i]
        line = line.split(",")
        run = line[1]
        camcol = line[2]
        field = line[3]
        ra_ = line[4]
        dec_ = line[5]
        field_lst.append([run, camcol, field])

    return field_lst


In [25]:
def make_sdss(galaxy_name, pgc, ra, dec, size, band):

    pgc_band_root = "."
    cmd = "mkdir " + pgc_band_root + "/tmp"
    xcmd(cmd, True)

    cmd = (
        "mv " + pgc_band_root + "/frame-" + band + "*.fits " + pgc_band_root + "/tmp/."
    )
    xcmd(cmd, True)

    ###########################################################################
    with cd(pgc_band_root):
        fout = open("montage.csh", "w")
        montage = """
      rm *tbl
      rm -rf projected
      mkdir  projected
      mImgtbl tmp rimages.tbl
      mProjExec -p tmp rimages.tbl tmp.hdr projected stats.tbl
      mImgtbl projected pimages.tbl
      mAdd -p projected pimages.tbl tmp.hdr tmp.fits
      mConvert -b -32 tmp.fits tmp.fits
      rm *area*fits
      rm *tbl
      rm -rf projected
      """
        fout.write(montage + "\n")
        fout.close()
        
#         if size > 1.1:
#             size = 1.1

        naxis = int(6 * size * 3600 / 0.396)
        crpix = naxis / 2
        hdr = ""
        hdr += "SIMPLE = T" + "\n"
        hdr += "BITPIX = -32" + "\n"
        hdr += "NAXIS = 2" + "\n"
        hdr += "NAXIS1 = " + str(naxis) + "\n"
        hdr += "NAXIS2 = " + str(naxis) + "\n"
        hdr += "CTYPE1 = 'RA---TAN'" + "\n"
        hdr += "CTYPE2 = 'DEC--TAN'" + "\n"
        hdr += "CRVAL1 = " + str(ra) + "\n"
        hdr += "CRVAL2 = " + str(dec) + "\n"
        hdr += "CRPIX1 = " + str(crpix) + "\n"
        hdr += "CRPIX2 = " + str(crpix) + "\n"
        hdr += "CDELT1 = -1.1E-04" + "\n"
        hdr += "CDELT2 = 1.1E-04" + "\n"
        hdr += "CROTA2 = 0.000000" + "\n"
        hdr += "EQUINOX = 2000.0" + "\n"
        hdr += "BSCALE = 1" + "\n"
        hdr += "BZERO = 0" + "\n"
        hdr += "EXPTIME = 1.0" + "\n"

        hdr += "MAGZERO =                 22.5 / Magnitude zeropoint\n"
        hdr += "BUNIT   = 'nanomaggy'          / 1 nanomaggy = 3.631e-6 Jy\n"
        hdr += f"FILTERX = '{band}       '           / Filter short name\n"
        hdr += "SURVEY  = 'SDSS DR12'\n"
        hdr += "FLAVOR  = 'science '           / Flavor of this run\n"
        hdr += "SYSTEM  = 'FK5     '           / System of the TCC coordinates (e.g., mean)\n"
        hdr += "EQUINOX =              2000.00 /\n"
        hdr += "ORIGIN  = 'SDSS    '\n"
        hdr += "TELESCOP= '2.5m    '\n"

        hdr += "OBJECT = " + galaxy_name + "\n"
        hdr += "PGC = " + f"PGC{pgc}" + "\n"
        hdr += "HISTORY = 'email: <ehsan@ifa.hawaii.edu>'" + "\n"

        hdr += "END" + "\n"

        fout = open("tmp.hdr", "w")
        fout.write(hdr)
        fout.close()

        xcmd("csh montage.csh", True)
        xcmd("mv tmp.fits " + galaxy_name + "_" + band + ".fits", True)
        xcmd("rm montage.csh", True)
        xcmd("rm tmp.hdr", True)
        xcmd("rm -rf tmp", True)


In [29]:
def get_sdss(galaxy_name, pgc, ra, dec, size, repository):

    fields = getSDSSfields(ra, dec, size)

    folder = repository + "tmp"
    if os.path.isdir(folder):
        xcmd("rm -rf " + folder, True)
    xcmd("mkdir " + folder, True)

    if not os.path.isdir(repository + galaxy_name):
        xcmd("mkdir " + repository + galaxy_name, True)

    for band in ["u", "g", "r", "i", "z"]:  # , "g", "r", "i", "z"

        for i in range(len(fields)):
            print(" ")
            print(
                " # "
                + str(i + 1)
                + "/"
                + str(len(fields))
                + "  ... obj: "
                + galaxy_name
                + "  band: "
                + band
            )

            getSDSSfiles(fields[i], band, folder)

        with cd(folder):
            make_sdss(galaxy_name, pgc, ra, dec, size, band)

        xcmd("mv " + folder + "/*fits " + repository + galaxy_name, True)
        xcmd("gzip " + repository + galaxy_name + "/*fits ", True)

In [30]:

galaxies = [
"IC3025",
"IC3032",
"IC3487",
"IC3586",
"N4458",
"N4472",
"N4489",
"N4649",
]


In [31]:
# for galaxy in ["IC1919", "IC2006", "N1344", "N1374", "N1375"]:
for galaxy in galaxies:


    pgc, objName, ra, dec, logd25 = load_info(galaxy)
    
    d25 = 10**logd25
    size = 0.1 * d25 / 60  # deg
    
    print(pgc, objName, ra, dec, logd25)
    
    get_sdss(galaxy, pgc, ra, dec, size, './SDSS/')

38726 IC3025 182.5962083333333 10.188555555555556 0.86

  
 SELECT
 fieldID,
 run, 
 camCol, 
 field,
 ra, 
 dec,
 run,
 rerun 
 FROM Field   
  WHERE ra BETWEEN 182.4513211413183 and 182.7410955253483 and dec BETWEEN 10.116111959548057 and 10.260999151563055

rm -rf ./SDSS/tmp

mkdir ./SDSS/tmp

mkdir ./SDSS/IC3025
 
 # 1/1  ... obj: IC3025  band: u

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/5973/3/frame-u-005973-3-0155.fits.bz2

bzip2 -d frame-u-005973-3-0155.fits.bz2

cp frame-u-005973-3-0155.fits frame-u-005973-3-0155.tmp.fits

mv frame-u-005973-3-0155.tmp.fits frame-u-005973-3-0155.fits

mkdir ./tmp

mv ./frame-u*.fits ./tmp/.

csh montage.csh

mv tmp.fits IC3025_u.fits

rm montage.csh

rm tmp.hdr

rm -rf tmp

mv ./SDSS/tmp/*fits ./SDSS/IC3025

gzip ./SDSS/IC3025/*fits 
 
 # 1/1  ... obj: IC3025  band: g

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/5973/3/frame-g-005973-3-0155.fits.bz2

bzip2 -d frame-g-005973-3-0155.fits.bz2

cp frame-g-005973-


bzip2 -d frame-u-003525-2-0221.fits.bz2

cp frame-u-003525-2-0221.fits frame-u-003525-2-0221.tmp.fits

mv frame-u-003525-2-0221.tmp.fits frame-u-003525-2-0221.fits
 
 # 5/5  ... obj: IC3487  band: u

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3525/2/frame-u-003525-2-0222.fits.bz2

bzip2 -d frame-u-003525-2-0222.fits.bz2

cp frame-u-003525-2-0222.fits frame-u-003525-2-0222.tmp.fits

mv frame-u-003525-2-0222.tmp.fits frame-u-003525-2-0222.fits

mkdir ./tmp

mv ./frame-u*.fits ./tmp/.

csh montage.csh

mv tmp.fits IC3487_u.fits

rm montage.csh

rm tmp.hdr

rm -rf tmp

mv ./SDSS/tmp/*fits ./SDSS/IC3487

gzip ./SDSS/IC3487/*fits 
 
 # 1/5  ... obj: IC3487  band: g

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3903/2/frame-g-003903-2-0047.fits.bz2

bzip2 -d frame-g-003903-2-0047.fits.bz2

cp frame-g-003903-2-0047.fits frame-g-003903-2-0047.tmp.fits

mv frame-g-003903-2-0047.tmp.fits frame-g-003903-2-0047.fits
 
 # 2/5  ... obj: IC3487  band: g

wget https:/


bzip2 -d frame-u-003064-4-0051.fits.bz2

cp frame-u-003064-4-0051.fits frame-u-003064-4-0051.tmp.fits

mv frame-u-003064-4-0051.tmp.fits frame-u-003064-4-0051.fits
 
 # 4/9  ... obj: IC3586  band: u

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/4/frame-u-003804-4-0208.fits.bz2

bzip2 -d frame-u-003804-4-0208.fits.bz2

cp frame-u-003804-4-0208.fits frame-u-003804-4-0208.tmp.fits

mv frame-u-003804-4-0208.tmp.fits frame-u-003804-4-0208.fits
 
 # 5/9  ... obj: IC3586  band: u

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/4/frame-u-003804-4-0209.fits.bz2

bzip2 -d frame-u-003804-4-0209.fits.bz2

cp frame-u-003804-4-0209.fits frame-u-003804-4-0209.tmp.fits

mv frame-u-003804-4-0209.tmp.fits frame-u-003804-4-0209.fits
 
 # 6/9  ... obj: IC3586  band: u

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/4/frame-u-003804-4-0210.fits.bz2

bzip2 -d frame-u-003804-4-0210.fits.bz2

cp frame-u-003804-4-0210.fits frame-u-003804-4-0210.tmp.fi


bzip2 -d frame-i-003064-4-0050.fits.bz2

cp frame-i-003064-4-0050.fits frame-i-003064-4-0050.tmp.fits

mv frame-i-003064-4-0050.tmp.fits frame-i-003064-4-0050.fits
 
 # 3/9  ... obj: IC3586  band: i

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/4/frame-i-003064-4-0051.fits.bz2

bzip2 -d frame-i-003064-4-0051.fits.bz2

cp frame-i-003064-4-0051.fits frame-i-003064-4-0051.tmp.fits

mv frame-i-003064-4-0051.tmp.fits frame-i-003064-4-0051.fits
 
 # 4/9  ... obj: IC3586  band: i

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/4/frame-i-003804-4-0208.fits.bz2

bzip2 -d frame-i-003804-4-0208.fits.bz2

cp frame-i-003804-4-0208.fits frame-i-003804-4-0208.tmp.fits

mv frame-i-003804-4-0208.tmp.fits frame-i-003804-4-0208.fits
 
 # 5/9  ... obj: IC3586  band: i

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/4/frame-i-003804-4-0209.fits.bz2

bzip2 -d frame-i-003804-4-0209.fits.bz2

cp frame-i-003804-4-0209.fits frame-i-003804-4-0209.tmp.fi

 
 # 1/8  ... obj: N4458  band: g

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3836/5/frame-g-003836-5-0254.fits.bz2

bzip2 -d frame-g-003836-5-0254.fits.bz2

cp frame-g-003836-5-0254.fits frame-g-003836-5-0254.tmp.fits

mv frame-g-003836-5-0254.tmp.fits frame-g-003836-5-0254.fits
 
 # 2/8  ... obj: N4458  band: g

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3836/5/frame-g-003836-5-0255.fits.bz2

bzip2 -d frame-g-003836-5-0255.fits.bz2

cp frame-g-003836-5-0255.fits frame-g-003836-5-0255.tmp.fits

mv frame-g-003836-5-0255.tmp.fits frame-g-003836-5-0255.fits
 
 # 3/8  ... obj: N4458  band: g

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3836/5/frame-g-003836-5-0256.fits.bz2

bzip2 -d frame-g-003836-5-0256.fits.bz2

cp frame-g-003836-5-0256.fits frame-g-003836-5-0256.tmp.fits

mv frame-g-003836-5-0256.tmp.fits frame-g-003836-5-0256.fits
 
 # 4/8  ... obj: N4458  band: g

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3836/5/

...


bzip2 -d frame-z-003031-6-0465.fits.bz2

cp frame-z-003031-6-0465.fits frame-z-003031-6-0465.tmp.fits

mv frame-z-003031-6-0465.tmp.fits frame-z-003031-6-0465.fits
 
 # 124/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3031/6/frame-z-003031-6-0466.fits.bz2

bzip2 -d frame-z-003031-6-0466.fits.bz2

cp frame-z-003031-6-0466.fits frame-z-003031-6-0466.tmp.fits

mv frame-z-003031-6-0466.tmp.fits frame-z-003031-6-0466.fits
 
 # 125/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3031/6/frame-z-003031-6-0467.fits.bz2

bzip2 -d frame-z-003031-6-0467.fits.bz2

cp frame-z-003031-6-0467.fits frame-z-003031-6-0467.tmp.fits

mv frame-z-003031-6-0467.tmp.fits frame-z-003031-6-0467.fits
 
 # 126/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3031/6/frame-z-003031-6-0468.fits.bz2

bzip2 -d frame-z-003031-6-0468.fits.bz2

cp frame-z-003031-6-0468.fits frame-z-003031-6-04


cp frame-z-003805-3-0033.fits frame-z-003805-3-0033.tmp.fits

mv frame-z-003805-3-0033.tmp.fits frame-z-003805-3-0033.fits
 
 # 151/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3805/3/frame-z-003805-3-0034.fits.bz2

bzip2 -d frame-z-003805-3-0034.fits.bz2

cp frame-z-003805-3-0034.fits frame-z-003805-3-0034.tmp.fits

mv frame-z-003805-3-0034.tmp.fits frame-z-003805-3-0034.fits
 
 # 152/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/1/frame-z-003804-1-0212.fits.bz2

bzip2 -d frame-z-003804-1-0212.fits.bz2

cp frame-z-003804-1-0212.fits frame-z-003804-1-0212.tmp.fits

mv frame-z-003804-1-0212.tmp.fits frame-z-003804-1-0212.fits
 
 # 153/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3804/1/frame-z-003804-1-0213.fits.bz2

bzip2 -d frame-z-003804-1-0213.fits.bz2

cp frame-z-003804-1-0213.fits frame-z-003804-1-0213.tmp.fits

mv frame-z-003804-1-0213.tmp


bzip2 -d frame-z-003805-2-0019.fits.bz2

cp frame-z-003805-2-0019.fits frame-z-003805-2-0019.tmp.fits

mv frame-z-003805-2-0019.tmp.fits frame-z-003805-2-0019.fits
 
 # 179/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3805/2/frame-z-003805-2-0020.fits.bz2

bzip2 -d frame-z-003805-2-0020.fits.bz2

cp frame-z-003805-2-0020.fits frame-z-003805-2-0020.tmp.fits

mv frame-z-003805-2-0020.tmp.fits frame-z-003805-2-0020.fits
 
 # 180/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3805/2/frame-z-003805-2-0021.fits.bz2

bzip2 -d frame-z-003805-2-0021.fits.bz2

cp frame-z-003805-2-0021.fits frame-z-003805-2-0021.tmp.fits

mv frame-z-003805-2-0021.tmp.fits frame-z-003805-2-0021.fits
 
 # 181/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3805/2/frame-z-003805-2-0022.fits.bz2

bzip2 -d frame-z-003805-2-0022.fits.bz2

cp frame-z-003805-2-0022.fits frame-z-003805-2-00


cp frame-z-003064-1-0064.fits frame-z-003064-1-0064.tmp.fits

mv frame-z-003064-1-0064.tmp.fits frame-z-003064-1-0064.fits
 
 # 206/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/1/frame-z-003064-1-0065.fits.bz2

bzip2 -d frame-z-003064-1-0065.fits.bz2

cp frame-z-003064-1-0065.fits frame-z-003064-1-0065.tmp.fits

mv frame-z-003064-1-0065.tmp.fits frame-z-003064-1-0065.fits
 
 # 207/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/1/frame-z-003064-1-0066.fits.bz2

bzip2 -d frame-z-003064-1-0066.fits.bz2

cp frame-z-003064-1-0066.fits frame-z-003064-1-0066.tmp.fits

mv frame-z-003064-1-0066.tmp.fits frame-z-003064-1-0066.fits
 
 # 208/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/1/frame-z-003064-1-0067.fits.bz2

bzip2 -d frame-z-003064-1-0067.fits.bz2

cp frame-z-003064-1-0067.fits frame-z-003064-1-0067.tmp.fits

mv frame-z-003064-1-0067.tmp


bzip2 -d frame-z-003064-3-0056.fits.bz2

cp frame-z-003064-3-0056.fits frame-z-003064-3-0056.tmp.fits

mv frame-z-003064-3-0056.tmp.fits frame-z-003064-3-0056.fits
 
 # 234/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/3/frame-z-003064-3-0057.fits.bz2

bzip2 -d frame-z-003064-3-0057.fits.bz2

cp frame-z-003064-3-0057.fits frame-z-003064-3-0057.tmp.fits

mv frame-z-003064-3-0057.tmp.fits frame-z-003064-3-0057.fits
 
 # 235/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/3/frame-z-003064-3-0058.fits.bz2

bzip2 -d frame-z-003064-3-0058.fits.bz2

cp frame-z-003064-3-0058.fits frame-z-003064-3-0058.tmp.fits

mv frame-z-003064-3-0058.tmp.fits frame-z-003064-3-0058.fits
 
 # 236/247  ... obj: N4649  band: z

wget https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/3064/3/frame-z-003064-3-0059.fits.bz2

bzip2 -d frame-z-003064-3-0059.fits.bz2

cp frame-z-003064-3-0059.fits frame-z-003064-3-00